# Лекция 6. Алгоритмы на графах

- Алгоритм Дейкстры
- A* (A-star)

# Алгоритм Дейкстры

Данный алгоритм позволяет найти кратчайший путь от одной вершины "A" до всех остальных. У него есть одно существенное ограничение - **длина ребер должна быть неотрицательная**.

Алгоритм состоит из следующих шагов:

- **Инициализация:** каждой вершине ставится метка расстояния от вершины "A", равная бесконечности (это эквивалентно отсутствию информации о пути). Метка для вершины "А", естественно, равна нулю. Все вершины графа отмечаются, как непосещенные. Формируем пул (открытых) вершин, вершин которые нужно проверить. Помещаем в этот пул вершину "А".

- **Ход работы**
  1. Если пул пуст. Заканчиваем работу.
  2. Забираем вершину с наименьшей меткой расстояния из пула вершин.
  3. Добавляются в пул все непосещенные вершины, соединенные с выбранной вершинной. Если сумма метки расстояния текущей вершины и длины ребра к соседней вершине меньше метки этой соседней вершины, то устанавливаем метку соседней вершины равную этой сумме. 
  4. Помечаем выбранную вершину посещенной.
  5. Повторить с пункта 1.

<img src="https://cdn.programiz.com/sites/tutorial2program/files/dj-1.png">

<img src="https://cdn.programiz.com/sites/tutorial2program/files/dj-2.png">

<img src="https://cdn.programiz.com/sites/tutorial2program/files/dj-3.png">

<img src="https://cdn.programiz.com/sites/tutorial2program/files/dj-4.png">

<img src="https://cdn.programiz.com/sites/tutorial2program/files/dj-5.png">

<img src="https://cdn.programiz.com/sites/tutorial2program/files/dj-6.png">

<img src="https://cdn.programiz.com/sites/tutorial2program/files/dj-7.png">

<img src="https://cdn.programiz.com/sites/tutorial2program/files/dj-8.png">

# Метро

Воспользуемся картой метро: https://mskof.ru/

In [124]:
import requests
import json

resp = requests.get("https://mskof.ru/json.json")

data = json.loads(resp.text)

In [128]:
data.keys()

dict_keys(['stations', 'lines', 'vertices'])

In [129]:
stations = data['stations']
stations = {
    k: v['n'] for k, v in stations.items()
}

In [156]:
for k, v in stations.items():
    if v in ['Беломорская', 'Войковская']:
        print(k, v)

220 Войковская
223 Беломорская


In [135]:
lines = data['lines']
lines = {
    k: v['n'] for k, v in lines.items()
}

In [184]:
class Node:
    def __init__(self, id, *, name):
        self.id = id
        self.name = name
        
        self.to_nodes = {}
        
        self.distance = None
        self.from_node = None
        
    def path(self, nodes):
        result = [(self.id, self.name)]
        current = self
        while current := nodes.get(current.from_node, None):
            result.append((current.id, current.name))
        return result
        
    def __repr__(self):
        r = f'< [{self.id:>5}]({self.distance if self.distance is not None else -1:3}) {self.name} - '
        for n in sorted(self.to_nodes):
            w = self.to_nodes[n]
            r += f'{n}: {w}, '
        r += '>'
        return r
    
n = Node('150', name="Станция")
n.to_nodes['151a'] = 2
n

< [  150]( -1) Станция - 151a: 2, >

In [185]:
vertices = data['vertices']

nodes = {}
for k, v in vertices.items():
    node = Node(k, name=stations.get(v['sid'], 'UNKNOWN'))
    nodes[k] = node
    for obj in v['e']:
        node.to_nodes[obj['to']] = obj['w']
    #print(node)

In [186]:
for node in nodes.values():
    node.distance = None
    node.from_node = None

pool = {'223b'}
nodes['223b'].distance = 0
visited = set()

while pool:
    tmp = sorted(pool, key=lambda x: nodes[x].distance if nodes[x].distance is not None else 9999999999)
    id_node = tmp[0]
    pool.remove(id_node)
    
    node = nodes[id_node]
    
    for n, w in node.to_nodes.items():
        if n in visited:
            continue
            
        path = node.distance + w
        neighbour = nodes[n]
        if neighbour.distance is None or neighbour.distance > path:
            neighbour.distance = path
            neighbour.from_node = node.id
        pool.add(neighbour.id)
    
    visited.add(id_node)        

In [191]:
nodes['150b'], nodes['150b'].path(nodes)

(< [ 150b]( 74) Коммунарка - >,
 [('150b', 'Коммунарка'),
  ('151b', 'Ольховая'),
  ('152b', 'Прошкино'),
  ('153b', 'Филатов Луг'),
  ('101b', 'Саларьево'),
  ('102b', 'Румянцево'),
  ('103b', 'Тропарёво'),
  ('104b', 'Юго-Западная'),
  ('105b', 'Проспект Вернадского'),
  ('106b', 'Университет'),
  ('107b', 'Воробьёвы горы'),
  ('108b', 'Спортивная'),
  ('109b', 'Фрунзенская'),
  ('110b', 'Парк культуры'),
  ('111b', 'Кропотнинская'),
  ('112b', 'Библиотека имени Ленина'),
  ('113b', 'Охотный Ряд'),
  ('213b', 'Театральная'),
  ('214b', 'Тверская'),
  ('215b', 'Маяковская'),
  ('216b', 'Белорусская'),
  ('217b', 'Динамо'),
  ('218b', 'Аэропорт'),
  ('219b', 'Сокол'),
  ('220b', 'Войковская'),
  ('221b', 'Водный стадион'),
  ('222b', 'Речной вокзал'),
  ('223b', 'Беломорская')])

# A*

Данный подход можно рассматривать, как оптимизацию алгоритма Дейкстры. Данный алгоритм позволяет найти оптимальный путь до указанной точки, использую оценочную эвристику.

- **Инициализация:** Формируем пул вершин (набор вершин, которые нужно просматривать). Помещаем в этот пул стартовую вершину.
- **Ход работы:**
   1. Если значение целевой вершины $f(x)$ меньше значения этой функции для любой вершины в пуле, то останавливаемся. Либо же, если пул пустой - тоже.
   2. Для каждой вершины в пуле вычисляем функцию $f(x) = g(x) + h(x)$. $g(x)$ - стоимость пути из стартовой вершины до текуущей. $h(x)$ - эвристика, оценивает на глаз стоимость о текущей вершины до целевой (фактически задает приоритет выбора вершин). Из пула забирается вершина с наименьшим значением.
   3. Выбранная вершина "раскрывается", то есть все непосещенные вершины добавляются в пул вершин.
   4. Вершина помечается посещенной.

<img src="https://www.101computing.net/wp/wp-content/uploads/A-Star-Search-Algorithm-Step-1.png">

<img src="https://www.101computing.net/wp/wp-content/uploads/A-Star-Search-Algorithm-Step-2.png">

<img src="https://www.101computing.net/wp/wp-content/uploads/A-Star-Search-Algorithm-Step-3.png">

<img src="https://www.101computing.net/wp/wp-content/uploads/A-Star-Search-Algorithm-Step-4.png">

<img src="https://www.101computing.net/wp/wp-content/uploads/A-Star-Search-Algorithm-Step-5.png">

<img src="https://www.101computing.net/wp/wp-content/uploads/A-Star-Search-Algorithm-Step-6.png">

<img src="https://www.101computing.net/wp/wp-content/uploads/A-Star-Search-Algorithm-Step-7.png">

В живую это выглядит так

<img src="https://upload.wikimedia.org/wikipedia/commons/5/5d/Astar_progress_animation.gif">

# Пример

In [8]:
import requests
import json

resp = requests.get("https://mskof.ru/json.json")

data = json.loads(resp.text)

In [9]:
data.keys()

dict_keys(['stations', 'lines', 'vertices'])

In [10]:
stations = data['stations']
stations = {
    k: v['n'] for k, v in stations.items()
}

In [11]:
lines = data['lines']
lines = {
    k: v['n'] for k, v in lines.items()
}

In [12]:
coords = {}
with open("coord.txt") as f:
    for line in f:
        line = line.strip()
        if line == "": continue
        k, x, y = line.split()
        coords[k] = (float(x), float(y))

In [20]:
class Node:
    def __init__(self, id, *, name):
        self.id = id
        self.name = name
        self.to_nodes = {}
        
        self.g = None # расстояние от начального до этого узла
        self.h = None # значение эвристики    
        self.from_node = None
        
    @property
    def f(self):
        return self.g + self.h
        
    def path(self, nodes):
        result = [(self.id, self.name)]
        current = self
        while current := nodes.get(current.from_node, None):
            result.append((current.id, current.name))
        return result
        
    def __repr__(self):
        tmp = f'{self.f if self.g is not None and self.h is not None else -1:3}'
        r = f'< [{self.id:>5}]({self.g if self.g is not None else -1:3} ' +\
            f'/ {self.h if self.h is not None else -1:3} = {tmp}) {self.name} - '
        for n in sorted(self.to_nodes):
            w = self.to_nodes[n]
            r += f'{n}: {w}, '
        r += '>'
        return r
    
n = Node('150', name="Станция")
n.to_nodes['151a'] = 2
n

< [  150]( -1 /  -1 =  -1) Станция - 151a: 2, >

In [26]:
import math

def makeH(target: tuple):
    def tmp(pos: tuple):
        return math.sqrt(
            (target[0] - pos[0])**2
            +
            (target[1] - pos[1])**2
        )
    return tmp

In [47]:
vertices = data['vertices']

nodes = {}
for k, v in vertices.items():
    node = Node(k, name=stations.get(v['sid'], 'UNKNOWN'))
    nodes[k] = node
    for obj in v['e']:
        node.to_nodes[obj['to']] = obj['w']
    #print(node)

In [48]:
h = makeH(coords['150'])

for node in nodes.values():
    node.g = None
    node.from_node = None
    node.h = h(coords.get(node.id[:-1], (0, 0)))
    # убираем направление пути
    if node.id[:-1] not in coords:
        print("WARNING: coords " + node.id)

pool = {'223b'}
nodes['223b'].g = 0
target = {'150a', '150b'}
visited = set()

while pool:
    node = None
    best = None
    for id in pool:
        f = nodes[id].f
        if best is None or best > f:
            best = f
            node = nodes[id]

    found = False
    for tid in target:
        tnode = nodes[tid]
        if tnode.g is None:
            continue
        if best > tnode.f:
            found = True
    if found:
        break
            
    for n, w in node.to_nodes.items():
        if n in visited:
            continue
            
        path = node.g + w
        neighbour = nodes[n]
        if neighbour.g is None or neighbour.g > path:
            neighbour.g = path
            neighbour.from_node = node.id
        pool.add(neighbour.id)
    
    pool.remove(node.id)
    visited.add(node.id)

In [39]:
nodes['150a']

< [ 150a]( -1 / 0.0 =  -1) Коммунарка - 151a: 2, >

In [46]:
nodes['150b']

< [ 150b]( 78 / 0.0 = 78.0) Коммунарка - >

In [45]:
nodes['150b'].path(nodes)

[('150b', 'Коммунарка'),
 ('151b', 'Ольховая'),
 ('152b', 'Прошкино'),
 ('153b', 'Филатов Луг'),
 ('101b', 'Саларьево'),
 ('102b', 'Румянцево'),
 ('103b', 'Тропарёво'),
 ('104b', 'Юго-Западная'),
 ('105b', 'Проспект Вернадского'),
 ('106b', 'Университет'),
 ('107b', 'Воробьёвы горы'),
 ('108b', 'Спортивная'),
 ('109b', 'Фрунзенская'),
 ('110b', 'Парк культуры'),
 ('111b', 'Кропотнинская'),
 ('112b', 'Библиотека имени Ленина'),
 ('914b', 'Боровицкая'),
 ('915b', 'Чеховская'),
 ('214b', 'Тверская'),
 ('215b', 'Маяковская'),
 ('216b', 'Белорусская'),
 ('217b', 'Динамо'),
 ('218b', 'Аэропорт'),
 ('219b', 'Сокол'),
 ('220b', 'Войковская'),
 ('221b', 'Водный стадион'),
 ('222b', 'Речной вокзал'),
 ('223b', 'Беломорская')]